# Données déséquilibrées
Dans la pratique, les problèmes de classification qu'ils soient bianires ou multiclasses se retrouvent face à des *datasets* déséquilibrés où une ou plusieurs classes sont présentes en effectifs sensiblement inférieur comparé à la ou les classes majoritaires. Dans la suite de ce document, on se place dans le cas de la classification binaire où un jeu de donné se répartit entre une classe dite majoritaire et une autre classe dite minoritaire.

Un fort déséquilibre des classes pose différents problèmes aux méthodes de classification supervisée. Par exemple:
* Les algorithmes font généralement l'hypothèse que les deux classes sont équilibrées (dit autrement, les erreurs faites par l'algorithme ne sont pas pondérés différemment suivant la classe). Si ce n'est pas le cas, il faut que le modèle en tienne compte (en général en introduisant une pondération à certain un moment de la procédure d'entraînement) sous peine d'obtenir un modèle en général biaisé en faveur de la classe majoritaire.
* Mauvaise performances en généralisation: si l'effectif de la classe minoritaire est trop faible, il risque de juste ne pas être possible d'apprendre une "bonne" frontière de décision (d'autant plus vrai que la dimensionnalité est élevée).
* Contraintes en validation et instabilité/incertitude et significativité des métriques: si l'effectif de la classe minoritaire est trop faible, on risque d'avoir trop peu de représentants de celle-ci dans chaque *fold* de la *cross-validation* avec pour conséquence un apprentissage et des métriques de performance instables.
* Certaines métriques deviennent inadaptées car faisant implicitement l'hypothèse d'un *dataset* équilibré : la classe minoritaire y contribue donc très peu et risque donc d'être négligée par l'algorithme qui va se biaiser en faveur de la classe majoritaire. Ex: l'*accuracy*. Dans le cas d'un *dataset* déséquilibré à 99/1 pour les classes 0/1, un classificateur retournant tout le temps 0 aura une *accuracy* de 99%.

## *Resampling*  (ou *class rebalancing techniques*)
Quand collecter plus de données n'est pas possible, une solution peut être de rééquilibrer le *dataset* par rééchantillonnage en veillant à ne pas modifier la distribution à l'intérieur de chaque classe (ce que tous les algorithmes ne garantissent pas). On peut à cette fin utiliser des méthodes de sous-échantillonnage de la classe majoritaire (*under-sampling*), de sur-échantillonnage de la classe minoritaire (*over-sampling*), ou les deux en combinaison.

On a pas à se limiter à un rééchantillonnage aléatoire qui finalement est assez pauvre et limitant. Il existe en effet tout un ensemble de techniques qui permettent 1) de rééquilibrer le dataset 2) d'améliorer sa séparabilité.

### Avertissement sur les *class rebalancing techniques*
Quelques remarques préliminaires importantes au sujet des *class rebalancing techniques*: 
* Comme toute opération consistant à transformer le *dataset*, celle-ci doit se faire **après** avoir séparé les données en *train* et *test set* **sous peine de *leakage***.
* Les *class rebalancing techniques* (notamment celles visant à augmenter le *dataset*, i.e. l'*over-sampling*) présentent un risque de modifier la distribution du *training set*. Le *dataset* rééquilibré peut ne plus être représentatif du *dataset* original et dans ce cas défavorable, les concepts appris par le modèle peuvent s'en retrouver différents/biaisés (*concept drift*). On se retrouve également avec un écart entre notre *training set* et *test set* (ce dernier étant représentatif des donnés **avant** rééquilibrage) rendant délicat l'interprétation des métriques de performance du modèle. Par exemple, on fait souvent l'hypothèse qu'un *downsampling* de la classe majoritaire a un impact limité, notamment du point de vue de la modification de la distribution de celle-ci. Toutefois préserver la distribution n'est pas tout. Les SVM par exemple n'ont finalement besoin que des points "frontaliers" de chaque classe qu'ils utiliseront comme vecteurs de support. Un *downsampling* qui viendrait ainsi supprimer trop de ces points limites peut avoir un impact significatif sur la fonction finalement apprise et les performances du modèle.
* En validation, veiller à ce qu'il n'y ait pas d'écart de représentativité entre chaque *fold* pour une bonne interprétabilité et stabilité des résultats (cf. `sklearn.model_selection.StratifiedKFold`).
* La présence de données catégorielles impose l'utilisation de méthodes spécifiques (ou au moins en interdit certaines).

### *Under-sampling* (ou *down-sampling*)
La solution la plus simple reste le *random under-sampling* de la classe majoritaire qui rééquilibre le *dataset* sans modifier la distribution de la classe échantillonnée mais au prix d'une perte importante de données qu'on ne peut pas toujours se permettre. Afin de ne pas avoir à renoncer à utiliser une part potentiellement très importantes des données, la libraire `imblearn` (cf. plus bas) propose une utilisation intelligente des *ensemble methods* (*random forest*, *boosting*) dans laquelle chaque *weak learner* est entrainé sur un *dataset* sous-échantillonné aléatoirement différent. Toutes les observations de la classe majoritaire sont ainsi potentiellement utilisées.

Remarque: Il existe en plus de la méthode aléatoire (`RandomUnderSampler`) toute une variété de méthode permettant de *down-sample* le *dataset*. Cf. `imblearn.under_sampling`. On peut citer notamment:
* `imblearn.under_sampling.ClusterCentroids` qui *down-sample* la classe majoritaire en s'aidant d'un $k$-means: un ensemble de points de la classe majoritaire est remplacé par son barycentre. Noter que les représentants de la classe majoritaire après *down-sampling* sont générés et non sélectionnés.
* `imblearn.under_sampling.NearMiss` qui regroupe différentes stratégies de sélection de membres de la classe majoritaire basée sur des notions de distance moyenne (à chaque stratégie correspond une définition de distance) à la classe majoritaire.

### *Over-sampling*
L'*over-sampling* consiste à rééquilibrer (à "augmenter") le *dataset* en augmentant la population de la classe majoritaire. Une première approche naïve consiste à rééchantillonner avec remise la classe majoritaire (cf. `imblearn.over_sampling.RandomOverSampler`). Cette approche est cependant peu efficace (mais a l'avantage de pouvoir s'appliquer quel que soit le type de données). Les algorithmes d'*over-sampling* vont consister à "créer" de nouvelles observations "synthétiques". Les principaux (d'ailleurs implémentés dans `imblearn`) sont le [SMOTE](https://arxiv.org/pdf/1106.1813.pdf) (*Synthetic Minority Oversampling TEchnique*) de Chawla et al. (2002) et [ADASYN](https://sci2s.ugr.es/keel/pdf/algorithm/congreso/2008-He-ieee.pdf) (*Adaptive Synthetic*) de He et al. (2008).

#### SMOTE
SMOTE et ADASYN reposent globalement tous les deux sur le même algorithme:
* Pour chaque membre de la classe minoritaire $x_{i}$ on va créer un nombre de nouveaux échantillons $g_i$ avec $\sum_{i}g_i$ correspondant au nombre d'échantillons synthétiques nécessaires à l'atteinte de l'objectif pour arriver au ratio minoritaire/majoritaire demandé. On va alors un nombre $g_i$ de fois:
    * Récupérer les membres $x_{i,r}$ de la même classe parmi ses $k$ plus proches voisins,
    * Sélectionner aléatoirement un $x_{i,r}$,
    * Générer un nouvel échantillon en le prenant sur le segment reliant $x_{i}$ à $x_{i,r}$ : $x_{new}=x_{i}+\lambda (x_{i,r} - x_{i})$ avec $\lambda$ un nombre aléatoire pris dans $[0, 1]$.

SMOTE est cependant de nature à créer des artefacts assez gênants qui peuvent considérablement modifier la distribution de la classe minoritaires si celle-ci présente des *outliers* où n'est pas convexe par exemple. Dans le cas où la classe minoritaire présente des *outliers* par exemple, si pour un $x_i$ donné un *outlier* figure parmi les plus proches voisins et est sélectionné pour la génération d'échantillons, on peut se retrouver avec une trainée de nouvelles observations jusqu'à assez loin en dehors de la distribution "normale" de la classe minoritaire, voire recoupant la distribution d'une autre classe.

#### *Borderline* SMOTE
Une amélioration connue du SMOTE est le [*borderline* SMOTE](https://sci2s.ugr.es/keel/keel-dataset/pdfs/2005-Han-LNCS.pdf) de Han et al. (2005) qui établit une distinction entre les représentants de la classe minoritaires: 
* Si les $k$ plus proches voisins de $x_i$ sont tous de la classe majoritaire, alors il est considéré comme du bruit (NOISE) et ne pourra servir à la génération d'échantillons. 
* Si la majorité des $k$ plus proches voisins de $x_i$ sont de la classe minoritaire, alors il est considéré comme SAFE et ne sera pas non plus utilisé pour la génération d'échantillons. 
* Dernier cas, si la majorité mais pas la totalité des $k$ plus proches voisins de $x_i$ sont de la classe majoritaire, alors il est considéré comme "dangereux" (DANGER). Seuls les points classés DANGER seront utilisés pour la génération d'échantillons dans le SMOTE *borderline*.

Cette version de l'algorithme est nommée *bordeline-1*, il existe une légère variante appelée *borderline-2* où les éléments de la classe majoritaire parmi les $k$ plus proches voisins sont sélectionnables pour la génération d'échantillons avec $\lambda$ seulement pris dans $[0, 0.5]$, les échantillons générés l'étant alors plus "près" de la classe majoritaire. 

Le *borderline* SMOTE ne va par construction renforcer la densité de la distribution de la classe minoritaire qu'au niveau de ses marges (en atténuant les écueils du SMOTE "standard") ce qui peut suivant la méthode de classification choisie avoir un impact en termes de *concept drift*. On peut par exemple imaginer qu'un classificateur à marge type SVM sera moins impacté qu'un autre algorithme.

Il existe de nombreuses autres variantes du SMOTE, voir notamment `imblearn.over_sampling`. 

#### ADASYN
ADASYN est une variante du SMOTE "standard", où le nombre $g_i$ d'échantillons générés à l'aide des voisins de $x_i$ est proportionnel à la part de membres de la classe majoritaire dans les $k$ plus proches voisins de $x_i$. L'idée rapproche ADASYN du SMOTE *borderline* qui dans les deux cas visent en priorité les membres "difficiles à apprendre" de la classe minoritaire (par opposition aux membres dont les plus proches voisins sont tous de la classe minoritaire qui seront à priori plus faciles à discriminer).  

Remarques: 
* Surveiller le ratio observations "synthétiques" sur observations originales. 
* Ces méthodes souffrent potentiellement des mêmes limitations que les méthodes "plus proches voisins" (*curse of dimensionality*, choix de la distance, etc.).

## Adaptation des métriques utilisées
Comme illustré plus haut, l'utilisation de l'*accuracy* n'est pas recommandée sauf si on est proche d'un équilibre entre les classes. On lui préfèrera la *balanced accuracy* définie comme la moyenne des *recalls* obtenus sur chaque classe et dont on comprend qu'elle a plus de sens dans le cas d'un *dataset* déséquilibré.

Par construction, la courbe ROC n'est pas sensible à un déséquilibre éventuel entre classes et on recommande alors de préférer l'utilisation de la courbe *Precision-Recall* (PR) qui par l'utilisation de la *precision* devient sensible à un éventuel déséquilibre. Un classificateur peut en effet avoir une performance (AUC) honorable du point de vue de la courbe ROC mais catastrophique du point de vue de la courbe PR. Cf. notes sur les métriques.

## Adaptation des algorithmes utilisés
La plupart des algorithmes ne supportent pas le cas déséquilibré par défaut et l'oublier risque de conduire à des classificateurs fortement biaisés en faveur de la classe majoritaire:
* Les algorithmes minimisant explicitement une fonction de coût pénalisent (ex: régression logistique) par défaut de la même manière une erreur faite sur la classe majoritaire et une erreur faite sur la classe minoritaire. Introduire une pondération (en général l'argument `class_weight` dans `sklearn`) permet de corriger le biais.
* Dans le cas des CART (arbres de décision), les critères de *split* usuels font par défaut l'hypothèse d'une répartition équilibrée des classes.
* Dans le cas d'autres algorithmes l'adaptation au déséquilibre (qui finit toujours par prendre la forme d'une pondération) peut être relativement spécifique (ex: SVM). 

## Autres options
### Reformulation du problème: Détection d'anomalies
Si le *dataset* est très déséquilibré et qu'il semble difficile d'établir une structure pour la classe minoritaire. Une possibilité est de changer d'approche (finalement à la suite d'une **mauvaise qualification du problème de départ**) et de voir le problème comme de la détection d'*outliers*. `sklearn` propose à cette date quatre algorithmes dédiés à cette catégorie de problèmes:
* `sklearn.svm.OneClassSVM`
* `sklearn.ensemble.IsolationForest`
* `sklearn.neighbors.LocalOutlierFactor`
* `sklearn.covariance.EllipticEnvelope`

`sklearn` établit une distinction entre *outlier detection* et *novelty detection*. Pris au sens strict la première approche se contente de détecter les anomalies dans un *dataset* donné là où la seconde vise à apprendre une frontière de décision permettant ensuite d'assigner une nouvelle observation à la classe des *inliers* ou des *outliers* (le point jusqu'alors inconnu classé comme *outlier* est désigné comme *novelty*). Les méthodes de la seconde catégorie appartiennent également à la première catégorie. Cette distinction de comportement ne concerne finalement ici que `sklearn.neighbors.LocalOutlierFactor` qui est la seule méthode à ne pas véritablement apprendre de frontière de décision mais qui se contente de calculer un score (elle ne calcule que la déviation de densité locale d'un point par rapport à la densité locale moyenne de ses $k$ plus proches voisins). La méthode peut toutefois être utilisée en *novelty detection* moyennant l'activation d'une option.

Remarque: Les methode d'*outlier detection* peuvent être utilisées pour nettoyer/débruiter un *dataset*.

### Reformulation du problème: *Clustering*
Si voir le problème comme un problème de *clustering* fonctionne (se pose ensuite le problème de rattacher une observation à un cluster lors de la prédiction), l'approche supervisée doit sans doute pouvoir fonctionner. A voir si cela est particulièrement adapté au besoin. Attention, il est possible que certaines méthodes de *clustering* souffrent (entre autres choses) de possibles déséquilibres entre les tailles des clusters à découvrir.

Exemple: Utiliser des *gaussian mixtures* pour apprendre les distributions de chacune des deux classes plutôt que de chercher une frontière de décision.

### Diminution du nombre de classes
Comme pour les variables catégorielles présentant un trop grand nombre de niveaux à faibles population, on peut tirer partie de rassembler les membres de plusieurs (petites) classes au sein d'une même classe ou d'une classe plus peuplée.

## Package `imblearn`
[`imblearn`](https://imbalanced-learn.readthedocs.io/en/stable/user_guide.html) ou `imbalanced-learn` est un [projet compatible](https://github.com/scikit-learn-contrib/imbalanced-learn) avec `sklearn` regroupant des utilitaires permettant d'effectuer des opérations de *resampling* de jeux de données déséquilibrés.  

Les modèles de `imblearn` sont des `Estimator` présentant sur le modèle des `Estimator` de `sklearn` deux méthodes `fit` et `sample` et une méthode permettant de faire les deux à la fois `fit_resample`.

```python
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.svm import LinearSVC

sampler = RandomUnderSampler(random_state=42)
clf = LinearSVC()

X_res, y_res = sample.fit_resample(X, y)
clf.fit(X_res, y_res)
```

`imblearn` propose un analogue à l'objet `sklearn.Pipeline.pipeline`:

```python
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.svm import LinearSVC

pca = PCA(n_components=3)
sampler = SMOTE(random_state=42)
clf = LinearSVC()
pipeline = make_pipeline(pca, sampler, clf)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# fit_transform classique
pipeline.fit(X_train, y_train)
y_hat = pipeline.predict(X_test)

# Grid search
param_range = range(1, 11)
train_scores, validation_scores = validation_curve(pipeline, X_train, y_train, 
                                                   param_name="sampler__k_neighbors", 
                                                   param_range=param_range,
                                                   cv=3, scoring="precision")
```